In [350]:
from pathlib import Path
import numpy as np
from enum import Enum
from typing import Tuple
from dataclasses import dataclass

In [351]:
def parse_map(lines):
    width = max(len(l) for l in lines)
    M = [list(l + ' ' * (width-len(l))) for l in lines]
    return np.array(M)
        
def parse_inst(insts):
    cur = ''
    out = []
    for c in insts:
        if c in ['L','R']:
            if len(cur) > 0:
                out.extend([int(cur),c])
            else:
                out.append(c)
            cur = ''
        else:
            cur = cur + c
    if len(cur) > 0:
        out.append(int(cur))
    return out




In [357]:
def corners(TL, c0, c1):
    TL = TL
    
    TR = list(TL)
    TR[c0] = 1 - TR[c0]
    TR = tuple(TR)

    BL = list(TL)
    BL[c1] = 1 - BL[c1]
    BL = tuple(BL)

    BR = list(TL)
    BR[c0] = 1 - BR[c0]
    BR[c1] = 1 - BR[c1]
    BR = tuple(BR)

    return [TL, TR, BL, BR]

In [360]:
corners((1,1,0), 1, 2)

[(1, 1, 0), (1, 0, 0), (1, 1, 1), (1, 0, 1)]

In [407]:
def parse_cube(lines):
    sz = int(len(lines[0])/3)
    M = np.zeros((6,sz,sz), dtype='U1')
    k = 0
    N = {}
    for i in range(0,len(lines), sz):        
        for j in range(0, len(lines[i]), sz):
            if lines[i][j] != ' ':
                M[k] = np.array([[c for c in l[j:j+sz]] for l in lines[i:i+sz]], dtype='U1')
                
                N[(i // sz, j//sz)] = k
                k +=1

                
                

    assert k == 6
    return M, N
        

In [502]:
class Point():
    def __init__(self, x,y,z):
        self._t = (x,y,z)

    def __add__(self, other):
        t = tuple(s+o for (s,o) in zip(self._t, other._t))
        return Point(*t)
    
    def __sub__(self, other):
        t = tuple(s-o for (s,o) in zip(self._t, other._t))
        return Point(*t)

    def __abs__(self):
        t = tuple(abs(s) for s in self._t)
        return Point(*t)

    def __repr__(self):
        return repr(self._t)

@dataclass
class Side:
    topleft : Point
    topright : Point
    bottomleft : Point
    bottomright : Point

    @property
    def coord_x(self):
        return abs(self.topright - self.topleft)

    @property
    def coord_y(self):
        return abs(self.topleft - self.bottomleft)

    @property
    def coord_other(self):
        return Point(1,1,1) - (self.coord_x + self.coord_y)

    def left(self):
        topright = self.topleft
        bottomright = self.bottomleft
        topleft = abs(self.topleft - self.coord_other)
        bottomleft = abs(self.bottomleft - self.coord_other)

        return Side(topleft, topright, bottomleft, bottomright)
        
    def right(self):
        topleft = self.topright
        bottomleft = self.bottomright
        topright = abs(self.topright - self.coord_other)
        bottomright = abs(self.bottomright - self.coord_other)

        return Side(topleft, topright, bottomleft, bottomright)

    def up(self):
        bottomleft = self.topleft
        bottomright = self.topright
        topleft = abs(self.topleft - self.coord_other)
        topright = abs(self.topright - self.coord_other)

        return Side(topleft, topright, bottomleft, bottomright)

    def down(self):
        topleft = self.bottomleft
        topright = self.bottomright
        bottomleft = abs(self.bottomleft - self.coord_other)
        bottomright = abs(self.bottomright - self.coord_other)

        return Side(topleft, topright, bottomleft, bottomright)


In [503]:
abs(Point(-3,0,7))

(3, 0, 7)

In [505]:
side = Side(Point(0,1,0), Point(1,1,0), Point(0,1,1), Point(1,1,1))
print(side.coord_y)
print(side.coord_x)
print(side.coord_other)

side.down()

(0, 0, 1)
(1, 0, 0)
(0, 1, 0)


Side(topleft=(0, 1, 1), topright=(1, 1, 1), bottomleft=(0, 0, 1), bottomright=(1, 0, 1))

In [510]:
side = Side(Point(0,0,0), Point(1,0,0), Point(0,1,0), Point(1,1,0))
side.down().left()


Side(topleft=(0, 0, 0), topright=(0, 1, 0), bottomleft=(0, 0, 1), bottomright=(0, 1, 1))

roll the die over the map

```
            (0,0,0) --- (1,0,0)
               |     x     |
               |y  (x,y)   |y
               |     x     |
(0,0,0) --- (0,1,0) --- (1,1,0) --- (1,0,0)
   |     y     |     x     |     y     |
   |z          |z  (x,z)   |z          |z
   |     y     |     x     |     y     |
(0,0,1) --- (0,1,1) --- (1,1,1) --- (1,0,1)
```


```
            (0,0,0) --- (1,0,0)
               |     x     |
               |y          |y
               |     x     |
(0,0,0) --- (0,1,0) --- (1,1,0)
   |     y     |     x     |
   |z          |z          |z
   |     y     |     x     |
(0,0,1) --- (0,1,1) --- (1,1,1) --- (1,1,0)
               |     x     |     z     |
               |y          |y         y|
               |     x     |     z     |
            (0,0,1) --- (1,0,1) --- (1,0,0)
```

In [409]:
M, I = Path(f'test/22.txt').read_text().rstrip().split('\n\n')
M, N = parse_cube(M.rstrip().split('\n'))
print(*N.items(), sep='\n')

((0, 2), 0)
((1, 0), 1)
((1, 1), 2)
((1, 2), 3)
((2, 2), 4)
((2, 3), 5)


In [411]:
M, I = Path(f'data/22.txt').read_text().rstrip().split('\n\n')
M, N = parse_cube(M.rstrip().split('\n'))
print(*N.items(), sep='\n')

((0, 1), 0)
((0, 2), 1)
((1, 1), 2)
((2, 0), 3)
((2, 1), 4)
((3, 0), 5)


In [402]:
def read(prefix='data', part=1):
    M, I = Path(f'{prefix}/22.txt').read_text().rstrip().split('\n\n')
    if part == 1:
        M = parse_map(M.rstrip().split('\n'))
    else:
        M = parse_cube(M.rstrip().split('\n'))
    return M, parse_inst(I)


In [324]:
M, I = read('test', part=2)

In [325]:
M.shape

(6, 4, 4)

In [316]:
class Facing(Enum):
    right = 0
    down = 1
    left = 2
    up = 3

    def turnR(self):
        val = (self.value +1) % 4
        return Facing(val)

    def turnL(self):
        val = (self.value -1) % 4
        return Facing(val)

    def turn(self, d):
        if d == 'L':
            return self.turnL()
        elif d == 'R':
            return self.turnR()
        else:
            raise Exception(f'unknown direction {d=}')

In [317]:
class Map:
    def __init__(self, M):
        self._M = M
        self.x = list(M[0]).index('.')
        self.y = 0
        self.facing = Facing.right
        self.__set_boundary()
    
    def __set_boundary(self):
        self.edgeL, self.edgeR = [], []
        for i in range(self._M.shape[0]):
            onmap = np.where(self._M[i] != ' ')[0]
            self.edgeL.append(onmap[0])
            self.edgeR.append(onmap[-1])

        self.edgeT, self.edgeB = [], []
        for i in range(self._M.shape[1]):
            onmap = np.where(self._M[:,i] != ' ')[0]
            self.edgeT.append(onmap[0])
            self.edgeB.append(onmap[-1])


    def move(self, inst):
        if inst in ['L', 'R']:
            self.facing = self.facing.turn(inst)
        else:
            inst = int(inst)
            self.forward(inst)

    def forward(self, num_steps : int):
        assert num_steps >= 0
        for _ in range(num_steps):
            moved = self.step()
            if not moved:
                return

    def step(self):
        match self.facing:
            case Facing.up:
                if self.y == self.edgeT[self.x]:
                    x_n, y_n = self.x, self.edgeB[self.x]
                else:
                    x_n, y_n = self.x, self.y - 1
            case Facing.down:
                if self.y == self.edgeB[self.x]:
                    x_n, y_n = self.x ,self.edgeT[self.x]
                else:
                    x_n, y_n = self.x, self.y + 1
            case Facing.left:
                if self.x == self.edgeL[self.y]:
                    x_n, y_n = self.edgeR[self.y], self.y
                else:
                    x_n, y_n = self.x - 1, self.y
            case Facing.right:
                if self.x == self.edgeR[self.y]:
                    x_n, y_n = self.edgeL[self.y], self.y
                else:
                    x_n, y_n = self.x + 1, self.y
            
        can_move = (self._M[y_n, x_n] == '.')
        if can_move:
            self.x, self.y = x_n, y_n
            #print(self.x, self.y, self.facing)
        return can_move


    def disp(self):
        print(*[''.join(r) for r in self._M], sep='\n')

    def password(self):
        return ((self.y+1) * 1000)  + ((self.x+1) * 4) + self.facing.value

In [318]:
M, I = read('test')
map = Map(M)
for inst in I:
    map.move(inst)
map.password()

6032

In [319]:
M, I = read()
map = Map(M)
for inst in I:
    map.move(inst)
map.password()

66292

In [320]:
sz = 4
M = np.zeros((sz+2,sz+2,sz+2), dtype='U1')

In [321]:
M.shape
# dx, dy, dz
# x, y, z

(6, 6, 6)

In [322]:
class Map2:
    def __init__(self, M, T):
        self._M = M
        self._T = T
        self.x = 0
        self.y = 0
        self.side = 0
        self.facing = Facing.right

    @property
    def size(self):
        return self._M.shape[1]    
    
    def move(self, inst):
        if inst in ['L', 'R']:
            self.facing = self.facing.turn(inst)
        else:
            inst = int(inst)
            self.forward(inst)

    def forward(self, num_steps : int):
        assert num_steps >= 0
        for _ in range(num_steps):
            moved = self.step()
            if not moved:
                return

    def transition_to(self):
        return self._T[self.side, self.facing](self.x, self.y)

    def step(self):
        s_t = self.transition_to(self.side, self.facing)
        match self.facing:
            case Facing.up:
                if self.y == 0:
                    s_n, x_n, y_n = s_t, self.x, self.size - 1
                else:
                    s_n, x_n, y_n = self.side, self.x, self.y - 1
            case Facing.down:
                if self.y == self.size - 1:
                    s_n, x_n, y_n = s_t, self.x , 0
                else:
                    s_n, x_n, y_n = self.side, self.x, self.y + 1
            case Facing.left:
                if self.x == 0:
                    s_n, x_n, y_n = s_t, self.size -1 , self.y
                else:
                    s_n, x_n, y_n = self.side, self.x - 1, self.y
            case Facing.right:
                if self.x == self.size - 1:
                    s_n, x_n, y_n = s_t, 0, self.y
                else:
                    s_n, x_n, y_n = self.side, self.x + 1, self.y
            
        can_move = (self._M[s_n, y_n, x_n] == '.')
        if can_move:
            self.size, self.x, self.y = s_n, x_n, y_n
        return can_move


    def disp(self):
        print(*[''.join(r) for r in self._M], sep='\n')

    def password(self):
        return ((self.y+1) * 1000)  + ((self.x+1) * 4) + self.facing.value